## Purpose
This notebook ingests and standardizes Community Planning Area boundaries for use in downstream spatial analysis.

## Run Order
1. Run all cells top-to-bottom
2. Outputs written to GetItDoneAnalysis.gdb
3. Downstream notebook: 02_getitdone_pipeline.ipynb

In [8]:
# Imports and evnironment setups
import os
import arcpy

arcpy.env.overwriteOutput = True

# --- Set project root folder 
PROJECT_ROOT = r"C:\Users\kris_\OneDrive - Kris Manske\Documents\Classes\BootcampGIS\Wildfire repositories on AWS\GetItDone"

RAW_DIR = os.path.join(PROJECT_ROOT, "data_raw")
WORK_DIR = os.path.join(PROJECT_ROOT, "data_working")
GDB_PATH = os.path.join(WORK_DIR, "GetItDoneAnalysis.gdb")

# Create folders if missing
for d in [RAW_DIR, WORK_DIR]:
    if not os.path.exists(d):
        os.makedirs(d)

# Create a file geodatabase if missing
if not arcpy.Exists(GDB_PATH):
    arcpy.management.CreateFileGDB(WORK_DIR, "GetItDoneAnalysis.gdb")

print("RAW_DIR:", RAW_DIR)
print("GDB_PATH:", GDB_PATH)

RAW_DIR: C:\Users\kris_\OneDrive - Kris Manske\Documents\Classes\BootcampGIS\Wildfire repositories on AWS\GetItDone\data_raw
GDB_PATH: C:\Users\kris_\OneDrive - Kris Manske\Documents\Classes\BootcampGIS\Wildfire repositories on AWS\GetItDone\data_working\GetItDoneAnalysis.gdb


In [9]:
# Point to CPA Shapefile
CPA_SHP = os.path.join(RAW_DIR, "communityPlanningAreasShp\cmty_plan_datasd.shp")  # change filename as needed

if not arcpy.Exists(CPA_SHP):
    raise FileNotFoundError(f"Can't find shapefile: {CPA_SHP}")

print("CPA shapefile found:", CPA_SHP)

CPA shapefile found: C:\Users\kris_\OneDrive - Kris Manske\Documents\Classes\BootcampGIS\Wildfire repositories on AWS\GetItDone\data_raw\communityPlanningAreasShp\cmty_plan_datasd.shp


In [10]:
import os
import arcpy

arcpy.env.overwriteOutput = True

# Set the workspace so ListFeatureClasses works
arcpy.env.workspace = GDB_PATH

# --- Import shapefile into the project GDB ---
# This tool imports and keeps the original name, so we’ll capture it explicitly.
arcpy.conversion.FeatureClassToGeodatabase([CPA_SHP], GDB_PATH)

# List feature classes now that workspace is set
imported = arcpy.ListFeatureClasses() or []
print("Feature classes in GDB:", imported)

# Find the imported FC by matching the shapefile base name
shp_base = os.path.splitext(os.path.basename(CPA_SHP))[0]
candidates = [fc for fc in imported if fc.lower() == shp_base.lower()]

if not candidates:
    # Fallback: if we can't match by name, assume the most recently created FC isn't easily known.
    raise RuntimeError(
        f"Could not find imported feature class matching shapefile name '{shp_base}'. "
        f"Imported FCs found: {imported}"
    )

imported_fc = os.path.join(GDB_PATH, candidates[0])
CPA_FC = os.path.join(GDB_PATH, "cpa")

# Rename to a consistent name "cpa"
if arcpy.Exists(CPA_FC):
    arcpy.management.Delete(CPA_FC)

arcpy.management.Rename(imported_fc, CPA_FC)
print("CPA feature class created:", CPA_FC)



Feature classes in GDB: ['cpa_prj', 'gid_drainage_points_lastmonth_with_cpa', 'gid_drainage_points_lastmonth_with_cpa_errors', 'gid_drainage_by_cpa_lastmonth', 'cmty_plan_datasd']
CPA feature class created: C:\Users\kris_\OneDrive - Kris Manske\Documents\Classes\BootcampGIS\Wildfire repositories on AWS\GetItDone\data_working\GetItDoneAnalysis.gdb\cpa


In [11]:
# Reproject the CPA features
# (Note - I know this is a bit of over-engineering because we are just doing point counts in the polygons
# and it will get reprojected to Web Mercator on the app - however, I wanted to practice reprojecting, and
# perhaps a future analysis will need distance measurement before I upload to AGOL)

TARGET_SR = arcpy.SpatialReference(2229)  # NAD_1983_StatePlane_California_VI_FIPS_0406_Feet

CPA_FC_PRJ = os.path.join(GDB_PATH, "cpa_prj")

if arcpy.Exists(CPA_FC_PRJ):
    arcpy.management.Delete(CPA_FC_PRJ)

arcpy.management.Project(CPA_FC, CPA_FC_PRJ, TARGET_SR)

print("Projected feature class:", CPA_FC_PRJ)
print("Projected SR:", arcpy.Describe(CPA_FC_PRJ).spatialReference.name)

# Remove WGS84 version so it doesn't get used by accident:
if arcpy.Exists(CPA_FC):
    arcpy.management.Delete(CPA_FC)


Projected feature class: C:\Users\kris_\OneDrive - Kris Manske\Documents\Classes\BootcampGIS\Wildfire repositories on AWS\GetItDone\data_working\GetItDoneAnalysis.gdb\cpa_prj
Projected SR: NAD_1983_StatePlane_California_V_FIPS_0405_Feet


In [12]:
fc = CPA_FC_PRJ  

# Count features
count = int(arcpy.management.GetCount(fc)[0])
print("CPA polygon count:", count)

# Check for null names in common fields (adjust field name once we confirm)
fields = [f.name for f in arcpy.ListFields(fc)]
print("Fields:", fields)

CPA polygon count: 61
Fields: ['OBJECTID_1', 'Shape', 'objectid', 'cpcode', 'cpname', 'acreage', 'Shape_Length', 'Shape_Area']


In [13]:
# Step to make sure fields don't have null or blank names
NAME_FIELD = "cpname"  # change based on actual fields

null_count = 0
with arcpy.da.SearchCursor(fc, [NAME_FIELD]) as cur:
    for (name,) in cur:
        if name is None or str(name).strip() == "":
            null_count += 1

print(f"Null/blank {NAME_FIELD} values:", null_count)


Null/blank cpname values: 0


In [14]:
# Also - kind of over-engineering, but again, just wanted to practice these steps.  Not really needed in final project, so this step could have been omitted.
#Schema standardization steps = Generate an ID for each record.  Calculate square mileage
fc = os.path.join(GDB_PATH, "cpa_prj")

# Rename fields (adjust original field names as needed)
field_map = {
    "cpname": "CPA_NAME"
}

existing_fields = [f.name for f in arcpy.ListFields(fc)]

for old, new in field_map.items():
    if old in existing_fields and new not in existing_fields:
        arcpy.management.AlterField(fc, old, new, new)

# Add CPA_ID if missing
if "CPA_ID" not in existing_fields:
    arcpy.management.AddField(fc, "CPA_ID", "LONG")

# Populate CPA_ID (simple sequential IDs)
with arcpy.da.UpdateCursor(fc, ["CPA_ID"]) as cur:
    i = 1
    for row in cur:
        row[0] = i
        cur.updateRow(row)
        i += 1

# Add area field (sq miles)
if "AREA_SQMI" not in existing_fields:
    arcpy.management.AddField(fc, "AREA_SQMI", "DOUBLE")

arcpy.management.CalculateGeometryAttributes(
    fc,
    [["AREA_SQMI", "AREA_GEODESIC"]],
    area_unit="SQUARE_MILES_US"
)

print("CPA schema standardized")


CPA schema standardized
